In [22]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.optim as optim

# Import for Hyperparameter optimization using Ax
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting

In [2]:

# Changing normal datatypes to tensors: 
# https://towardsdatascience.com/deep-learning-on-dataframes-with-pytorch-66b21be54ef6
# https://stackoverflow.com/questions/44617871/how-to-convert-a-list-of-strings-into-a-tensor-in-pytorch

# pytorch nn model
# https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/

In [3]:
# train = pd.read_csv('clean.csv')
# X = train.drop(['status_group'], axis=1)
# X_train, X_test, y_train, y_test =train_test_split(X, y,test_size=0.2, random_state=42,stratify=y)

In [4]:
# # train_ds = TensorDataset(X_train, y_train)
# train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

# test_ds = TensorDataset(X_test, y_test)
# test_dl = DataLoader(test_ds, batch_size=bs * 2)

In [5]:
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [6]:
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path)
        
# #       Change string values into float
#         labels = ['amount_tsh']
#         print("columns===" , df.columns.to_list())
#         df.drop(['amount_tsh'], inplace=True)
#         le = preprocessing.LabelEncoder()
#         targets = le.fit_transform(labels)
#         # targets: array([0, 1, 2, 3])

#         targets = torch.as_tensor(targets)
        # store the inputs and outputs
#         print(df.index)
        print(df.head())
        print('shape = ', df.shape)
        self.X = df.values[:, :-1]
#         print(self.X.head())
        self.y = df.values[:, -1]
        print(self.y)
        print(self.y)
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
 
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

In [7]:
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl
 
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
 
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc
 
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [8]:
path = 'clean-pt.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# print(train_dl.shape)


   id  amount_tsh  funder  gps_height  installer  longitude  latitude  basin  \
0   0          27    1634        1129       1713      13266     28993      3   
1   1          34    1501        2056       1576      26432      7686      6   
2   2          27     838        1129        930      35413     24830      8   
3   3           8     436        1717        201      42515     47836      5   
4   4          27     211        1129        228      35678     25765      8   

   region  district_code  ...  scheme_management  extraction_type  management  \
0      19              3  ...                  7                0           7   
1       3              4  ...                 12               12           7   
2       2              4  ...                  7                7           7   
3       6              5  ...                 10                3           9   
4       2              4  ...                  7                8           7   

   payment_type  water_quality  

In [9]:
# define the network
model = MLP(21)
# train the model
train_model(train_dl, model)
# evaluate the model
# make a single prediction (expect class=1)
# row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
# yhat = predict(row, model)
# print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))

In [10]:
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)


Accuracy: 0.541


In [21]:
## helper train/fit function

def train(model, parameterization, train_dl):
    optimizer = optim.SGD(model.parameters(), lr=parameterization["lr"], momentum=parameterization["momentum"])
    criterion = BCELoss()
    
    for epoch in range(2):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, (inputs, targets) in enumerate(train_dl, 0):

            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
            
            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
        
    return model

            
## helper function to evaluate the accuracy for the tested model
def evaluate(model, test_dl):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
        predictions, actuals = vstack(predictions), vstack(actuals)
        # calculate accuracy
        acc = accuracy_score(actuals, predictions)
        print('Accuracy of model:', acc)
        return acc

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

## helper function train-evaluate to pass as the function to be optimized
def train_evaluate(parameterization):
    model = MLP(21)
    model = train(model, parameterization, train_dl)
    return evaluate(model, test_dl)
    

In [12]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True},
        {"name": "momentum", "type": "range", "bounds": [0.0, 1.0]},
    ],
    evaluation_function=train_evaluate,
    objective_name='accuracy',
    total_trials=15
)

[INFO 11-07 17:35:05] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 11-07 17:35:05] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter momentum. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 11-07 17:35:05] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 5 trials, GPEI for subsequent trials]). Iterations after 5 will take longer to generate due to  model-fitting.
[INFO 11-07 17:35:05] ax.service.managed_loop: Started full optimization with 15 steps.
[INFO 11-07 17:35:05] ax.service.managed_loop: Running optimization trial 1...
[INFO 11-07 17:35:08] ax.service.managed_loop: Running optimizati

Accuracy of model: 0.078125


[INFO 11-07 17:35:10] ax.service.managed_loop: Running optimization trial 3...


Accuracy of model: 0.078125


[INFO 11-07 17:35:13] ax.service.managed_loop: Running optimization trial 4...


...

Accuracy of model: 0.5625


C:\Users\User\anaconda3\lib\site-packages\ax\modelbridge\torch.py:311: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

[INFO 11-07 17:35:44] ax.service.managed_loop: Running optimization trial 15...


Accuracy of model: 0.078125


C:\Users\User\anaconda3\lib\site-packages\ax\modelbridge\torch.py:311: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Accuracy of model: 0.5625


In [13]:
best_parameters

{'lr': 0.003067257245374468, 'momentum': 0.286615805731618}

In [14]:
values

({'accuracy': 0.5624999895828608},
 {'accuracy': {'accuracy': 4.788144985609108e-09}})

In [29]:
test_df = CSVDataset('clean_test-pt.csv')
# model.predict(test_df)
# result = test_df.apply(lambda row: model.predict(row), axis=1)
test = pd.DataFrame(read_csv('clean_test-pt.csv'))
result = test.apply(lambda row: predict(row,model), axis=1)

   Unnamed: 0     id  amount_tsh  funder  gps_height  installer  longitude  \
0           0  10144           9     175        1977        214       7525   
1           1  10319          12     250        1576        221       9370   
2           2   3416          16     620        1574       1076       6676   
3           3   9123          16     222         314        262      12109   
4           4   9953          28      73        1264         75       7190   

   latitude  basin  region  ...  population  scheme_management  \
0      9392      0       8  ...         261                  2   
1     11209      5       0  ...         249                  6   
2      7494      0      18  ...         361                  6   
3      1843      7       7  ...         210                  6   
4       295      7      16  ...          52                  9   

   extraction_type  management  payment_type  water_quality  quantity  source  \
0                9           3             2         

TypeError: 'TorchModelBridge' object is not callable

In [17]:
test_model = MLP(21)
test_model

MLP(
  (hidden1): Linear(in_features=21, out_features=10, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=10, out_features=8, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=8, out_features=1, bias=True)
  (act3): Sigmoid()
)

In [18]:
model

In [20]:
train_model(train_dl,test_model)